# Nowcasting in High-dimension: an application

Chapter 10


In [1]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web

from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression, Lasso
from src.stats_tools import SparseGroupLasso, SVD
from sklearn.pipeline import Pipeline

# 0. Custom functions

## A. Data transformations

In [2]:
def create_lags(x, l_min: int = 0, l_max: int = 1):
    """
    create_lags:
        Returns everything with the lags.
    
    Args:
        x (pd.DataFrame): The dataframe to consider, indexed by the time.
        l_min (int): Minimum of lags to have.
        l_max (int): Maximum of lags to add.
        
    Return:
        A concatenated dataset with all the lags for all series.
    """
    lags = pd.concat({
        l: x.shift(l) for l in range(l_min, l_max+1)
    }, axis=1)
    return lags

def ts_standardize(x, halflife: int = 12, ignore_na: bool = True):
    """
    ts_standardize:

    Args:
        x (pd.DataFrame):
        halflife (int):
        ignore_na (bool):
    """
    return (
        x
        .sub(x.ewm(halflife=halflife, ignore_na=ignore_na).mean(), axis=1)
        .div(x.ewm(halflife=halflife, ignore_na=ignore_na).std(), axis=1)
    )

def legendre_polynomials(degree, a=0, b=1, jmax=None, X=None):
    """
    legendre_polynomials:
        Legendre polynomials shifted to [a,b]
        Source: https://github.com/jstriaukas/midasmlpy/blob/master/midasmlpy/src/midas_polynomials.py

    Args:
        degree (int): Degree of the polynomial.
        a (float): Lower shift value (i.e., default - 0).
        b (float): Upper shift value (i.e., default - 1).
        jmax (int): Number of high-frequency lags.
        X (ndarray): Optional evaluation grid vector.
    
    Return
        Psi (ndarray): Weight matrix with Legendre functions up to ``degree``.
    """
    if jmax is not None:
        X = np.linspace(start=0, stop=1, num=jmax)

    if X is None:
        raise ValueError("X is not provided. Either set X or set jmax.")

    n = len(X)
    P = np.ones([n, degree+2])
    Psi = np.ones([n, degree+1]) / np.sqrt(b-a)
    P[:, 1] = 2 * X / (b-a) - ((b+a) / (b-a))
    if degree > 0:
        for i in range(1, degree+1, 1):
            P[:, i+1] = ((2*i+1)/(i+1)) * P[:, 1] * P[:, i] - i/(i+1) * P[:, i-1]
            Psi[:, i] = np.sqrt((2*i + 1) / (b-a)) * P[:, i]
    return Psi


# 1. Load data

## A. Topic attention data

In [3]:
DATA_PATH = "http://www.structureofnews.com/data/download/Monthly_Topic_Attention_Theta.csv"
SELECTED_COLS = [
    i-2 for i in [43,37,20,2,42,15,35,6,7,16,38,181,71,73,11,17,40,77,119,142,145,147,167,170,176]
]

In [4]:
df_attention = (
    pd.read_csv(DATA_PATH)
    .assign(date = lambda x: pd.to_datetime(x["date"]))
    .set_index("date")
    .iloc[:, SELECTED_COLS]
    * 100
)
print(df_attention.shape)
print(df_attention.head())

(402, 25)
            Announce plan  Revised estimate  Record high  Natural disasters  \
date                                                                          
1984-01-01       0.467674          0.688088     0.480592           0.339301   
1984-02-01       0.424175          0.656877     0.417940           0.463997   
1984-03-01       0.408318          0.658685     0.444813           0.529381   
1984-04-01       0.392498          0.623401     0.448515           0.418133   
1984-05-01       0.412422          0.558719     0.420943           0.417861   

            Problems  Middle east  US defense   Profits       M&A  \
date                                                                
1984-01-01  0.538521     0.746824    0.810809  0.962193  0.458591   
1984-02-01  0.555107     1.085275    0.913313  0.895614  0.520514   
1984-03-01  0.510741     0.849796    0.952218  0.552926  0.611201   
1984-04-01  0.510228     0.768855    0.814239  1.093373  0.457250   
1984-05-01  0.545848  

## B. US GDP

In [5]:
df_gdp = web.DataReader(
    "GDPC1",
    "fred",
    pd.to_datetime("1980-01-01"),
    pd.to_datetime("2022-12-31"),
)
df_gdp = (
    df_gdp
    .div(df_gdp.shift(1))
    .pipe(lambda x: 100 * (x ** 4 - 1))
)
print(df_gdp.shape)
print(df_gdp.head())

(172, 1)
               GDPC1
DATE                
1980-01-01       NaN
1980-04-01 -7.990637
1980-07-01 -0.474542
1980-10-01  7.670986
1981-01-01  8.071104


## C. Chicago Fed National Activity Index (CFNAI)

In [6]:
df_cfnai = web.DataReader(
    "CFNAI",
    "fred",
    pd.to_datetime("1980-01-01"),
    pd.to_datetime("2022-12-31"),
)
print(df_cfnai.shape)
print(df_cfnai.head())

(516, 1)
            CFNAI
DATE             
1980-01-01   0.13
1980-02-01  -0.52
1980-03-01  -1.14
1980-04-01  -2.19
1980-05-01  -2.30


## D. Non-Farm Payrolls

In [7]:
df_nfp = web.DataReader(
    "PAYEMS",
    "fred",
    pd.to_datetime("1980-01-01"),
    pd.to_datetime("2022-12-31"),
)

df_nfp = (
    df_nfp
    .pipe(np.log)
    .diff(1)
    * 100
)
print(df_nfp.shape)
print(df_nfp.head())


(516, 1)
              PAYEMS
DATE                
1980-01-01       NaN
1980-02-01  0.091368
1980-03-01  0.122061
1980-04-01 -0.159478
1980-05-01 -0.473331


## E. Aruoba-Diebold-Scotti Business Conditions Index

In [8]:
ADS_PATH = "https://www.philadelphiafed.org/-/media/frbp/assets/surveys-and-data/ads/ads_index_most_current_vintage.xlsx"

df_ads = (
    pd.read_excel(ADS_PATH, names = ["date", "ADS_index"])
    .assign(date = lambda x: pd.to_datetime(x["date"].str.replace(':', '-')))
    .set_index("date")
)
print(df_ads.shape)
print(df_ads.head())

(23056, 1)
            ADS_index
date                 
1960-03-01  -0.579788
1960-03-02  -0.626985
1960-03-03  -0.671207
1960-03-04  -0.712469
1960-03-05  -0.750784


## F. Consolidating and transforming data

In [9]:
dastart = "1990-01-01" # Beginning of training period
daend_train = "2002-01-01" # End of training period

# Concatenate the data, resample at monthly frequency using latest value, and standardize using EMAs.
df_ = (
    pd.concat([df_gdp, df_cfnai, df_nfp, df_ads, df_attention], axis=1)
    .dropna(how="all")
    .resample('M').last()
    .loc[dastart:]
    .pipe(ts_standardize)
)

# 2. Training the model

In [10]:
X_cov_lags = create_lags(df_.drop("GDPC1", axis=1), l_max=9).ffill(limit=3) # Lags of covariates
y_lags = create_lags(df_[["GDPC1"]].resample('3M').last(), l_max=4) # Lags of outcome

Xy_ = pd.concat([y_lags, X_cov_lags], axis=1).dropna().swaplevel(axis=1).sort_index(axis=1) # Putting everything together and dropping NAs when outcome is not here.
y, X = Xy_[('GDPC1', 0)], Xy_.drop(('GDPC1', 0), axis=1) # Assign outcome and regressors.

In [14]:
group_dict = {
    g: np.where(X.columns.get_level_values(0) == g)[0].tolist() for g in X.columns.get_level_values(0).unique()
}

groups = list(group_dict.values())

estimators = {
    'SparseGroupLasso': Pipeline([
        ('regression', SparseGroupLasso(groups=groups, gamma=0.8, alpha=1, verbose=False, max_iter=1_000))
    ]),
    'SVD with Ridge rotation': Pipeline([
        ('svd', SVD(alpha=1e4)),
        ('ols', LinearRegression())
    ])

}

tscv = TimeSeriesSplit(n_splits = 5)

In [16]:
X_, y_ = X.values, y.values
for i, (train_index, test_index) in enumerate(tscv.split(X_)):
    print(f"Fold {i}:")
    print(f"  Train period: index={Xy_.index[train_index].min().date()} -> {Xy_.index[train_index].max().date()}")
    print(f"  Test period:  index={Xy_.index[test_index].min().date()} -> {Xy_.index[test_index].max().date()}")
    
    for name, est in estimators.items():
        est.fit(X_[train_index], y_[train_index])
        print(f"R2 {name}: {est.score(X_[test_index], y_[test_index]):.2f}")

Fold 0:
  Train period: index=1991-04-30 -> 1996-04-30
  Test period:  index=1996-07-31 -> 2000-07-31


ValueError: Complex data not supported
[0.15676661+0.j 0.15676661+0.j 0.15676661+0.j 0.15676661+0.j
 0.15676661+0.j 0.15676661+0.j 0.15676661+0.j 0.15676661+0.j
 0.15676661+0.j 0.15676661+0.j 0.15676661+0.j 0.15676661+0.j
 0.15676661+0.j 0.15676661+0.j 0.15676661+0.j 0.15676661+0.j
 0.15676661+0.j]


In [13]:
est.named_steps['svd'].rotation.shape

(284, 284)